In [1]:
import math
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

C:\Users\AttahiruJibril\AppData\Local\Temp\ipykernel_2756\1970053945.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



xy = pd.read_table(data, header=0)
new_col = []
for names in xy.columns:
    new_col.append(names.strip())
xy.columns = new_col
x, y = xy.loc[:, 'text'], xy.loc[:, ['EmotionalPolarity', 'Emotion', 'Empathy']]

tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(x), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda y: np.array(y)


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = get_tokenizer("basic_english")
class EEPD_Dataset(Dataset):
    def __init__(self, x, y):
        
        self.text = torch.tensor(x).to(device)
        self.label = torch.tensor(y).to(device)
        self.n_samples = x.shape[0]


    def __getitem__(self, index):
        return self.text[index], self.label[index]
    
    def __len__(self):
        return self.n_samples



In [16]:
data = 'data/WASSA23_conv_level_with_labels_train.tsv'

def yield_tokens(text_data):
    for text in text_data:
        yield tokenizer(text)

xy = pd.read_table(data, header=0)
new_col = []
for names in xy.columns:
    new_col.append(names.strip())
xy.columns = new_col
x, y = xy.loc[:, 'text'], xy.loc[:, ['EmotionalPolarity', 'Emotion', 'Empathy']]
y = np.array(y)



vocab = build_vocab_from_iterator(yield_tokens(x), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
train_x, train_y = [], []
for text, label in zip(x, y):
    x_tokens = vocab(tokenizer(text))
    y_label = np.array(label)
    train_x.append(x_tokens)
    train_y.append(y_label)



In [18]:
data = 'data/WASSA23_conv_level_with_labels_train.tsv'
dataset = EEPD_Dataset(train_x, train_y)
# dataloader = DataLoader(dataset=dataset, batch_size=1, shuffle=False)
# data_iter = iter(dataloader)

# for i in range(2):
#     f,l = next(data_iter)
# f,l
dataset[0][0]

ValueError: expected sequence of length 8 at dim 1 (got 15)

In [3]:
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xy = pd.read_table(data, header=0)
new_col = []
for names in xy.columns:
    new_col.append(names.strip())
xy.columns = new_col
x, y = xy.loc[:, 'text'], xy.loc[:, ['EmotionalPolarity', 'Emotion', 'Empathy']]
y = np.array(y)


tokenizer = get_tokenizer("basic_english")

# Tokenize sequences
tokenized_sequences = [tokenizer(text) for text in x]

# Convert tokenized sequences to tensors
tensor_sequences = [torch.tensor(tokens) for tokens in tokenized_sequences]

# Pad sequences
padded_sequences = pad_sequence(tensor_sequences, batch_first=True)

# Build vocabulary
vocab = build_vocab_from_iterator(yield_tokens(x), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Convert labels to numpy array
y = np.array(y)

# Process the data
train_x, train_y = [], []
max_length = padded_sequences.size(1)  # Get the maximum length after padding

for text, label in zip(x, y):
    x_tokens = torch.tensor(vocab(tokenizer(text))).to(device)
    
    # Pad the tokenized sequence to max_length
    x_tokens_padded = F.pad(x_tokens, (0, max_length - len(x_tokens)))
    
    y_label = torch.tensor(label).to(device)
    
    train_x.append(x_tokens_padded)
    train_y.append(y_label)

train_x = torch.stack(train_x)
train_y = torch.stack(train_y)

# Now train_x and train_y are tensors with padded sequences




ValueError: too many dimensions 'str'

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def collate_batch(text , label):
    label_list, text_list, offsets = [], [], [0]
    
    for  _text, _label in text , label:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [ ]:

class EEPD_Dataset(Dataset):
    def __init__(self):
        data = 'data/WASSA23_conv_level_with_labels_train.tsv'
        xy = pd.read_table(data, header=0)
        new_col = []
        for names in xy.columns:
            new_col.append(names.strip())
        xy.columns = new_col
        x, y = xy.loc[:, 'text'], xy.loc[:, ['EmotionalPolarity', 'Emotion', 'Empathy']]
        self.x = np.array(x)
        self.y = torch.from_numpy(np.array(y))
        self.n_samples = xy.shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples
    



In [ ]:
n_epochs = 8
n_samples = len(dataset)
n_iterations = math.ceil(n_samples/4)

print(n_samples, n_iterations)

for epoch in range(n_epochs):
    for i, (input, labels) in enumerate(dataloader):
        print(f'Epoch: {epoch+1}/{n_epochs} | step{i+1}/{n_iterations}')

- tokenization
- remove stop word and punctuatuons, numbers
- lematization
- vectorization

In [77]:
def word_preprocessor(sentence):
    stop_words = set(stopwords.words('english'))
    punctuations = set(string.punctuation)
    lem = WordNetLemmatizer().lemmatize
    sentence = word_tokenize(sentence)
    sentence = [word for word in sentence if word not in stop_words]
    sentence = [word for word in sentence if word not in punctuations]
    sentence_str = ' '.join(sentence)
    sentence = lem(sentence_str)
    return sentence #sentence